# Установка библиотеки bayoo-docx

In [8]:
# установка bayoo-docx
#!pip install bayoo-docx

In [19]:
# для поиска строки в файле используем thefuzz (нечеткий поиск строк)
#!pip install thefuzz[speedup]

## Необходимые import

In [1]:
# Необходимые импорты
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from thefuzz import fuzz
import ctypes

# Модуль ctypes нужен для определения полного имени пользователи.

C:\Anaconda3\envs\py33\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Создаем тестовый документ и наполняем

In [7]:
# создание пустого документа
doc = Document()

#добавляем абзацы
doc.add_paragraph('Первый абзац, первая страница')
doc.add_paragraph('Второй абзац, первая страница')
doc.add_paragraph('Третий абзац, первая страница')

# добавляем разрыв страницы
doc.add_page_break()

#добавляем абзацы на второй странице
doc.add_paragraph('Первый абзац, вторая страница')
doc.add_paragraph('Второй абзац, вторая страница')
doc.add_paragraph('Третий абзац, вторая страница')

# данные таблицы без названий колонок
items = (
    (1, 'первая строка', 'первая строка'),
    (2, 'вторая строка', 'вторая строка'),
    (3, 'третья строка', 'третья строка'),
)

# добавляем таблицу с одной строкой 
# для заполнения названий колонок
table = doc.add_table(1, len(items[0]))

# определяем стиль таблицы
table.style = 'Light Shading Accent 1'

# Получаем строку с колонками из добавленной таблицы
head_cells = table.rows[0].cells

# добавляем названия колонок
for i, item in enumerate(['первая колонка', 'вторая колонка', 'третья колонка']):
    p = head_cells[i].paragraphs[0]
    # название колонки
    p.add_run(item).bold = True
    # выравниваем посередине
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
# добавляем данные к существующей таблице
for row in items:
    # добавляем строку с ячейками к объекту таблицы
    cells = table.add_row().cells
    for i, item in enumerate(row):
        # вставляем данные в ячейки
        cells[i].text = str(item)
        
# сохраняем тестовый файл с которым будем работать       
doc.save('test.docx')

In [9]:
# содержание списка elements
for element in doc.elements:
    print(element)

# Сборк абзацев из файла в список

In [10]:
"""Функция для сбора все параграфов в файле, это необходимо т.к. файл может содержать такие структуры как paragraph, table... 
В структуре table (таблица) можно итерировать по ячейкам (cell). Cell так же содержат paragraph(абзацы), а это как раз таки то, 
что нам нужно (комментарии можно добавлять как к прогонам (run) так и к абзацам (paragraph))."""


def filter_element(document):
    """
    This function take all paragraphs in file.

    :param document: object of document
    :return: list of paragraphs in file.docx - document
    """
    res = []
    for element in document.elements:
        if 'paragraph' in str(element): 
            res.append(element)
        elif 'table' in str(element):
            for row in element.rows:
                for cell in row.cells:
                    res.append(cell.paragraphs)
    return res 

# Определение полного имени пользователя

In [11]:
def get_display_name():
    """
    This function return full name of user.
    out:
        string: full name of user
    """
    get_user_name_ex = ctypes.windll.secur32.GetUserNameExW
    name_display = 3
    size = ctypes.pointer(ctypes.c_ulong(0))
    get_user_name_ex(name_display, None, size)
    name_buffer = ctypes.create_unicode_buffer(size.contents.value)
    get_user_name_ex(name_display, name_buffer, size)
    return name_buffer.value

# Поиск строки и добавление комментария

In [15]:
def make_comment(text:str, paragraphs:list, user:str):
    """
    This function adds comments in docx files.
    :param text: the line we are looking for
    :param paragraphs: list of paragraphs to search for a string
    :param user: full name of user
    """
    for paragraph in paragraphs:
        # итерируем по абзацам и проверяем на тип 
        if type(paragraph) == list:
            # если список значит здесь ячейка таблицы
            text_in_table = [p.text for p in paragraph]
            text_in_table = ''.join(text_in_table)
            # соединяем абзацы и получаем целый текст не потеряв список обьектов Paragraph
            if len(text_in_table) >= len(text)-5:
                # проверяем, что длина абзаца как минимум меньше на 10 символов
                res = fuzz.partial_ratio(text.lower(), text_in_table.lower())
                # производим сравнение ячейки и строки которую ищем
                if res >= 97:
                    # если индекст выше 97 производим добавление комментария к пустому прогону добавленному в конец
                    p = paragraph[-1]
                    run = p.add_run()
                    run.add_comment('Строчка которую искали', author=user) # передали комментарий и имя пользователя
        else:
            # сюда попадают проверки для абзацев вне таблицы.
            if len(paragraph.text) >= len(text):
                res = fuzz.partial_ratio(text.lower(), paragraph.text.lower())
                if res >= 97:
                    paragraph.add_comment('Строчка которую искали', author=user) # комментарий добавляется к абзацу
 

#обьект документа
document = Document('test.docx')
# строчка которую ищем
text = "Первый абзац, первая страницы"
# список абзацев для поиска
paragraphs = filter_element(document)
# полное имя пользователя
name_of_user = get_display_name()
# вызываем функцию добавления комментарий
make_comment(text, paragraphs, name_of_user)
# сохраняем изменения в файл .docx
document.save('test с комментарием.docx')

# Комментарий к строке в таблице

In [16]:
#обьект документа
document = Document('test.docx')
# строчка которую ищем
text = "Первая строка"
# список абзацев для поиска
paragraphs = filter_element(document)
# полное имя пользователя
name_of_user = get_display_name()
# вызываем функцию добавления комментарий
make_comment(text, paragraphs, name_of_user)
# сохраняем изменения в файл .docx
document.save('test с комментарием.docx')

# Поиск номера страницы

In [17]:
def number_page(text:str, paragraphs:list):
    """
    This funcion find number page.

    :param text: string what we find
    :param paragraphs: list of paragraphs
    :return: pages
    """
    # список для сбора номеров страниц где встретилась строка
    pages = []
    # счетчик для номеров страниц
    number_page = 1
    for paragraph in paragraphs:
        if type(paragraph) == list:
            # проверка на наличие разрывов страниц внутри таблиц
            text_in_table = [p.text for p in paragraph]
            text_in_table = ''.join(text_in_table)
            for p in paragraph:
                for run in p.runs:
                    # проверка на мягкий разрыва страницы
                    if 'lastRenderedPageBreak' in run._element.xml:
                        number_page += 1
                    # проверка на жесткий разрыв страницы
                    elif 'w:br' in run._element.xml and 'type="page"' in run._element.xml:
                        number_page += 1
            # к этому моменту известен актуальный номер страницы
            if len(text_in_table) >= len(text)-10:
                res = fuzz.partial_ratio(text.lower(), text_in_table.lower())
                if res >= 97:
                    # если строчка найдена, добавляем в список номер страницы
                    pages.append(number_page)
        else:
            # проверка разрывов в абзацах вне таблиц
            for run in paragraph.runs:
                if 'lastRenderedPageBreak' in run._element.xml:
                    number_page += 1
                elif 'w:br' in run._element.xml and 'type="page"' in run._element.xml:
                    number_page += 1
            if len(paragraph.text) >= len(text):
                res = fuzz.partial_ratio(text.lower(), paragraph.text.lower())
                if res >= 97:
                    pages.append(number_page)
    return ', '.join(map(str, pages))


# создаем обьект document
document = Document('test.docx')
# снова собираем в список абзацы документа
paragraphs = filter_element(document)
# строка номер которой хотим найти
text = "Первая строка"
# вызываем функцию для поиска номера стриницы искомого текста.          
print(number_page(text, paragraphs))

2, 2


In [18]:
# Через запятую выводятся страницы в которых была обнаружена строчка.